# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 27 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [1]:
# !pip install torchsummaryX wandb --quiet

In [2]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

/home/zzy/miniconda3/envs/11785/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device:  cuda


In [3]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

In [5]:
# Configs:
DATA_ROOT = "/mnt/e/Workspace/IDL/Data/hw1/11-785-s24-hw1p2/"
MODEL_ROOT = "/mnt/e/Workspace/IDL/Models/hw1/11-785-s24-hw1p2/"

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [6]:
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
# !mkdir /root/.kaggle

# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"Replace this with your Kaggle Username","key":"Replace this with your kaggle API key"}')
#     # Put your kaggle username & key here

# !chmod 600 /root/.kaggle/kaggle.json

In [7]:
# commands to download data from kaggle
# !kaggle competitions download -c 11785-hw1p2-s24

# !unzip -qo /content/11785-hw1p2-s24.zip -d '/content'

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [8]:
# Dataset class to load train and validation data
from torch.nn.functional import one_hot
from torch.utils.data import Dataset, DataLoader
#TODO: dynamic padding
class AudioDataset(Dataset):
    def __init__(self, root=DATA_ROOT, phonemes = PHONEMES, context=0, partition= "train-clean-100", use_cmn=False): #TODO: make no-pad works (BF2042 meme huh?) 
        self.max_context_length = 1145 #Magic number
        self.set_context_length(context)
        self.phonemes   = phonemes
        
        self.num_phonemes = len(self.phonemes)
        
        self.mfccs, self.transcripts = self._init_data(f"{root}/{partition}", use_cmn=use_cmn)
        self.length = len(self.mfccs)
        
        self.mfccs = np.concatenate([np.zeros((self.max_context_length, 27)), self.mfccs, np.zeros((self.max_context_length, 27))], axis=0)
        
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        # given current context length, compute offset:
        lower = idx + self.max_context_length - self.context
        upper = idx + self.max_context_length + self.context + 1
            

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[lower:upper]
        
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # Reshape to 1d array

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[idx]) # Get the phoneme at the index

        return frames, phonemes
        
        
    def _init_data(self, root: str, use_cmn = False):
        self.mfcc_dir       = f"{root}/mfcc"
        self.transcript_dir = f"{root}/transcript"
        mfcc_names          = os.listdir(self.mfcc_dir)
        transcript_names    = os.listdir(self.transcript_dir)
        
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []
        for i in tqdm(range(len(mfcc_names))):
        #   Load a single mfcc
            mfcc        = np.load(f"{self.mfcc_dir}/{mfcc_names[i]}")
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            if use_cmn:
                mfcc = mfcc - np.mean(mfcc, axis=0)
        #   Load the corresponding transcript
            transcript  = np.load(f"{self.transcript_dir}/{transcript_names[i]}") 
            # Remove [SOS] and [EOS] from the transcript
            assert transcript[0] == '[SOS]' and transcript[-1] == '[EOS]'
            transcript = transcript[1:-1]
            #lookup phoneme index
            transcript = np.vectorize(self.phonemes.index)(transcript)
            assert len(mfcc) == len(transcript)
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)
            
        return np.concatenate(self.mfccs, axis=0), np.concatenate(self.transcripts, axis=0)
    
    def set_context_length(self, context):
        self.context = context
        
    def phoneme_reverse_lookup(self, idx: torch.tensor) -> str:
        return self.phonemes[idx]
         
         
val_data = AudioDataset(partition="dev-clean", context=5)

  0%|          | 12/2703 [00:00<00:23, 114.66it/s]

100%|██████████| 2703/2703 [00:17<00:00, 151.14it/s]


In [9]:
f, p = val_data[114514]
print(f.shape, p, val_data.phoneme_reverse_lookup(p))

torch.Size([297]) tensor(3) AH


In [10]:
from numpy import ndarray


class AudioTestDataset(AudioDataset):
    
    def _init_data(self, root: str, use_cmn):
        
        self.mfcc_dir = f"{root}/mfcc"

        mfcc_names = os.listdir(self.mfcc_dir)

        self.mfccs, self.transcripts = [], []
        
        for i in tqdm(range(len(mfcc_names))):
        #   Load a single mfcc
            mfcc        = np.load(f"{self.mfcc_dir}/{mfcc_names[i]}")
            transcript = np.array([0 for _ in range(len(mfcc))])
            
            assert len(mfcc) == len(transcript)
            
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)
            
        return np.concatenate(self.mfccs, axis=0), np.concatenate(self.transcripts, axis=0)
    
    def __getitem__(self, ind):
        return super().__getitem__(ind)[0]
    

test_data = AudioTestDataset(partition="test-clean", context=5)

100%|██████████| 2620/2620 [00:08<00:00, 292.86it/s]


In [11]:
f = test_data[114514]
print(f.shape)

torch.Size([297])


In [12]:
train_data = AudioDataset(partition="train-clean-100", context=5)

100%|██████████| 28539/28539 [03:16<00:00, 145.48it/s]


#  Parameters Configuration and Create Datasets

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [13]:
config = {
    'epochs'        : 50,
    'batch_size'    : 65536,
    'context'       : 25,
    'init_lr'       : 1e-3,
    'architecture'  : 'deep_mlp',
    'dropout'       : 0.1,
    'weight_decay'  : 1e-5,
    'scheduler_params'     : {'patience': 5, 'factor': 0.2, 'min-lr': 1e-7},
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
    'wandb_name': 'bs_65k_2_layer_mlp'
}


In [14]:
#TODO: Create a dataset object using the AudioDataset class for the training data
train_data.set_context_length(config['context'])

# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data.set_context_length(config['context'])

# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data.set_context_length(config['context'])

In [15]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 6,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  65536
Context        :  25
Input size     :  1377
Output symbols :  42
Train dataset samples = 36091157, batches = 551
Validation dataset samples = 1928204, batches = 30
Test dataset samples = 1934138, batches = 30


In [16]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([65536, 1377]) torch.Size([65536])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [17]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class LowCutoffNet(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(LowCutoffNet, self).__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(config['dropout']),
            torch.nn.Linear(256, 2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(config['dropout']),
            torch.nn.Linear(2048, output_size)
        )

    def forward(self, x):
        out = self.model(x)

        return out

In [18]:
from torch import nn
from torch.nn.modules import Module

class DynamicMlpNet(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_sizes, dropout_rate):
        super(DynamicMlpNet, self).__init__()
        self.layers = []
        for i, hs in enumerate(hidden_sizes):
            self.layers.extend(self._mlp_layer_provider(input_size, hs, dropout_rate))
            input_size = hs
        self.layers.append(nn.Linear(input_size, output_size)) # output
        self.model = nn.Sequential(*self.layers)
    
    def forward(self, x):
        out = self.model(x)
        return out
    
    def _mlp_layer_provider(self, input_size, hidden_size, dropout_rate) -> list[nn.Module]:
        return [
            nn.Linear(input_size, hidden_size),
            nn.LeakyReLU(),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(dropout_rate)
        ]
        
class NetV2(DynamicMlpNet):
    def __init__(self, input_size, output_size, hidden_sizes, dropout_rate):
        hidden_layers = np.array([1,2,4,2,1]) * 512
        return super().__init__(input_size, output_size, hidden_layers, dropout_rate)
    
    def _mlp_layer_provider(self, input_size, hidden_size, dropout_rate) -> list[Module]:
        return [
            nn.Linear(input_size, hidden_size),
            nn.Mish(),
            nn.Dropout(dropout_rate)
        ]
    

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [19]:
INPUT_SIZE  = (2*config['context'] + 1) * 27 # Why is this the case?
model       = LowCutoffNet(INPUT_SIZE, len(train_data.phonemes)).to(device)
# Some upstream dependency bug in torchsummaryX, using torchinfo instead: 
import torchinfo
torchinfo.summary(model, input_size=(config['batch_size'], INPUT_SIZE))
# summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 24 million parameters for HW1 (including ensembles)

Layer (type:depth-idx)                   Output Shape              Param #
LowCutoffNet                             [65536, 42]               --
├─Sequential: 1-1                        [65536, 42]               --
│    └─Linear: 2-1                       [65536, 256]              352,768
│    └─ReLU: 2-2                         [65536, 256]              --
│    └─Dropout: 2-3                      [65536, 256]              --
│    └─Linear: 2-4                       [65536, 2048]             526,336
│    └─ReLU: 2-5                         [65536, 2048]             --
│    └─Dropout: 2-6                      [65536, 2048]             --
│    └─Linear: 2-7                       [65536, 42]               86,058
Total params: 965,162
Trainable params: 965,162
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 63.25
Input size (MB): 360.97
Forward/backward pass size (MB): 1229.98
Params size (MB): 3.86
Estimated Total Size (MB): 1594.81

In [20]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr'], weight_decay=config['weight_decay'], ) #Defining Optimizer

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=config['scheduler_params']['patience'], 
    min_lr=config['scheduler_params']['min-lr'], 
    factor=config['scheduler_params']['factor'],
    verbose=True
)
# Recommended : Define Scheduler for Learning Rate,
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [21]:
torch.cuda.empty_cache()
gc.collect()

6

In [22]:
from torch.cuda.amp import GradScaler

def train(model, dataloader, optimizer, criterion, scheduler, logger, log_freq=100, use_amp=True):
    scaler = GradScaler()
    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    
    if use_amp:
        for i, (frames, phonemes) in enumerate(dataloader):
            ### Initialize Gradients
            optimizer.zero_grad()

            ### Move Data to Device (Ideally GPU)
            frames      = frames.to(device)
            phonemes    = phonemes.to(device)

            ### Forward Propagation
            with torch.autocast(device_type='cuda', dtype=frames.dtype):
                logits  = model(frames)
                ### Loss Calculation
               
                loss    = criterion(logits, phonemes)

            ### Backward Propagation
            scaler.scale(loss).backward()

            ### Gradient Descent
            scaler.step(optimizer)
            scaler.update()
            
            scheduler.step(loss)

            tloss   += loss.item()
            # print(torch.argmax(logits, dim= 1), phonemes.shape, tloss)
            tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]
            
            # raise Exception("Shape mismatch")
            

            batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                                acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
            batch_bar.update()

            ### Release memory
            del frames, phonemes, logits
            if (i+1)%log_freq == 0:
                logger(i, tloss / (i+1), tacc*100 / (i+1))
            torch.cuda.empty_cache()
    else:
        for i, (frames, phonemes) in enumerate(dataloader):
            ### Initialize Gradients
            optimizer.zero_grad()

            ### Move Data to Device (Ideally GPU)
            frames      = frames.to(device)
            phonemes    = phonemes.to(device)

            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)
                

            ### Backward Propagation
            loss.backward()

            ### Gradient Descent
            optimizer.step()
            

            tloss   += loss.item()
            tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

            batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                                acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
            batch_bar.update()

            ### Release memory
            del frames, phonemes, logits
            if (i+1)%log_freq == 0:
                logger(i, tloss / (i+1), tacc*100 / (i+1))
            torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [23]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()
        

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [24]:
wandb.login(key="a07bacf1f6490c2d1a0d4e22dd08701319310f93") #API Key is in your wandb account, under settings (wandb.ai/settings)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: zzyatcmu (school_stuff). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/zzy/.netrc


True

In [25]:
MODEL_ROOT = "/mnt/e/Workspace/IDL/Models/hw1/11-785-s24-hw1p2/model_ver1_1"
if not os.path.exists(MODEL_ROOT):
    os.makedirs(MODEL_ROOT)

In [26]:
# # Create your wandb run
run = wandb.init(
    name    = '2layer_test', ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config, ### Wandb Config for your run
    dir = MODEL_ROOT ### Wandb local directory
)

In [27]:
# ### Save your model architecture as a string with str(model)
model_arch  = str(model)

# ### Save it in a txt file
arch_file   = open(f"{MODEL_ROOT}/model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

# ### log it in your wandb run with wandb.save()
wandb.save(arch_file.name)

wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


['/mnt/e/Workspace/IDL/Models/hw1/11-785-s24-hw1p2/model_ver1_1/wandb/run-20240206_040918-zbhd9won/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [28]:
def wandb_logger(iter, loss, acc):
    wandb.log({"train_loss": loss, "train_acc": acc})

In [29]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")


for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion, scheduler, logger=wandb_logger, log_freq=100, use_amp=False)
    scheduler.step(train_loss)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})
    

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
    torch.save(model.state_dict(), f"{MODEL_ROOT}/model_{epoch}.cpt")

### Finish your wandb run
run.finish()
# Baseline model 1st epoch:x``
#   Train Acc 51.7162%	Train Loss 1.6885	 Learning Rate 0.0010000
#	Val Acc 63.6595%	Val Loss 1.1859


Epoch 1/50


Train:   0%|          | 0/551 [00:00<?, ?it/s]

	Train Acc 51.7162%	Train Loss 1.6885	 Learning Rate 0.0010000
	Val Acc 63.6595%	Val Loss 1.1859

Epoch 2/50


	Train Acc 63.0104%	Train Loss 1.1954	 Learning Rate 0.0010000
	Val Acc 67.7324%	Val Loss 1.0390

Epoch 3/50


	Train Acc 66.3193%	Train Loss 1.0776	 Learning Rate 0.0010000
	Val Acc 69.8127%	Val Loss 0.9652

Epoch 4/50


...

	Train Acc 76.0858%	Train Loss 0.7449	 Learning Rate 0.0010000
	Val Acc 77.0365%	Val Loss 0.7167

Epoch 48/50


	Train Acc 76.1028%	Train Loss 0.7442	 Learning Rate 0.0010000
	Val Acc 77.0361%	Val Loss 0.7162

Epoch 49/50


	Train Acc 76.1244%	Train Loss 0.7437	 Learning Rate 0.0010000
	Val Acc 77.0585%	Val Loss 0.7157

Epoch 50/50


	Train Acc 76.1411%	Train Loss 0.7430	 Learning Rate 0.0010000
	Val Acc 77.0233%	Val Loss 0.7161


lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▄▅▆▆▇▇▇▇▇▇▇████████████████████████████
train_loss,█▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇███████████████████████
valid_loss,█▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,0.001
train_acc,76.14111
train_loss,0.74302
val_acc,77.02331
valid_loss,0.71609


# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [30]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.no_grad(): 

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            max_idxs = torch.argmax(logits, dim=1)
            
            predicted_phonemes = [test_loader.dataset.phoneme_reverse_lookup(max_idx) for max_idx in max_idxs]

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            test_predictions.extend(predicted_phonemes)
    return test_predictions

In [34]:
predictions = test(model, test_loader)

100%|██████████| 30/30 [01:03<00:00,  2.11s/it]


In [36]:
### Create CSV file with predictions
with open(f"{MODEL_ROOT}/submission_latest.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [33]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
# !kaggle competitions submit -c 11785-hw1p2-s24 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle